In [1]:
%load_ext autoreload
%matplotlib inline
%autoreload 2
%config Completer.use_jedi = True
import matplotlib as mpl
from matplotlib import pyplot as plt
mpl.rc("figure", dpi=100)
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore')
import scalp
from scalp.output import draw
import lmz
import numpy as np

In [2]:
scalp.test_config.scib_datapath

'/home/ikea/data/benchdata/'

In [ ]:
datasets = scalp.data.loaddata_scib(scalp.test_config.scib_datapath, 
                                    maxdatasets=3,  maxcells = 1000 )
dataset = datasets[-1] # this is the pancreatic

/home/ikea/.myconda/miniconda3/envs/scalp2/lib/python3.11/site-packages/anndata/_io/specs/registry.py:249: OldFormatWarning: Element '/layers' was written without encoding metadata.
  return self.callback(read_func, elem.name, elem, iospec=get_spec(elem))
/home/ikea/.myconda/miniconda3/envs/scalp2/lib/python3.11/site-packages/anndata/_io/specs/registry.py:249: OldFormatWarning: Element '/layers/counts' was written without encoding metadata.
  return self.callback(read_func, elem.name, elem, iospec=get_spec(elem))
/home/ikea/.myconda/miniconda3/envs/scalp2/lib/python3.11/site-packages/anndata/_io/utils.py:205: OldFormatWarning: Element '/obs/__categories/batch' was written without encoding metadata.
  return func(*args, **kwargs)
/home/ikea/.myconda/miniconda3/envs/scalp2/lib/python3.11/site-packages/anndata/_io/utils.py:205: OldFormatWarning: Element '/obs/batch' was written without encoding metadata.
  return func(*args, **kwargs)
/home/ikea/.myconda/miniconda3/envs/scalp2/lib/python3

# scanorama

In [ ]:
import scanpy as sc

def scanorama(dataset, dim = 2):
    dataset = scalp.pca.pca(dataset)
    dataset = scalp.mnn.scanorama(dataset)
    [ds.obsm.pop('umap', None)for ds in dataset]
    scalp.umapwrap.adatas_umap(dataset, label ='umap', from_obsm ='scanorama', dim= dim)
    return scalp.transform.stack(dataset)

st = scanorama(dataset)
sc.pl.umap(st, color=['batch', 'label'])

# UMAP ONLY 

In [ ]:
import scanpy as sc
def umaponly(dataset,dim=2):
    dataset = scalp.pca.pca(dataset)
    [ds.obsm.pop('umap', None)for ds in dataset]
    scalp.umapwrap.adatas_umap(dataset, label ='umap', from_obsm ='pca40', dim =dim)
    return scalp.transform.stack(dataset)
    
stack = umaponly(dataset)
sc.pl.umap(stack, color=['batch', 'label'])

In [ ]:
import scanpy as sc
def bbknn(dataset, dim = 2):
    dataset = scalp.pca.pca(dataset)
    [ds.obsm.pop('umap', None)for ds in dataset]
    dataset = scalp.mnn.bbknnwrap(dataset, dim = dim)
    stack = scalp.transform.stack(dataset)
    return stack

stack = bbknn(dataset)
sc.pl.umap(stack, color=['batch', 'label'])

# Combat 

In [ ]:
import scanpy as sc
def combat(dataset, dim = 2):
    [ds.obsm.pop('umap', None)for ds in dataset]
    dataset = scalp.mnn.combat(dataset)
    scalp.umapwrap.adatas_umap(dataset, label ='umap', from_obsm ='combat', dim =dim)
    return scalp.transform.stack(dataset)
stack= combat(dataset)
sc.pl.umap(stack, color=['batch', 'label'])

# SCALP

In [ ]:
import scanpy as sc
from scalp import graph as sgraph
from scipy.sparse import csr_matrix

def Scalp(dataset, dim = 2):
    parm = {'neighbors_total': 30, 'intra_neighbors_mutual': False,
            'neighbors_intra_fraction': .3, 'add_tree': False, "epsilon":1e-4,
                  'copy_lsa_neighbors': False, 
            'inter_outlier_threshold': 0.9, 
            'inter_outlier_probabilistic_removal': False}
    bestparm = {'add_tree': True, 'copy_lsa_neighbors': False, 'inter_outlier_probabilistic_removal': False, 
            'inter_outlier_threshold': 0.7433835019772639, 'intra_neighbors_mutual': True, 
                'neighbors_intra_fraction': 0.5689801219518802, 'neighbors_total': 31.0}
    
    # parm = {'add_tree': True, 'copy_lsa_neighbors': False, 'inter_outlier_probabilistic_removal': False,  'inter_outlier_threshold': 0.72, 'intra_neighbors_mutual': False, 'neighbors_intra_fraction': 0, 'neighbors_total': 1}
    dataset, grap = scalp.mkgraph(dataset,**parm)
    # neggraph = sgraph.negstuff(dataset,**parm)
    # scalp.umapwrap.graph_jax(dataset,(csr_matrix(grap), csr_matrix(neggraph)),label = 'umap', n_components = 2)
    
    plt.matshow(grap.todense())
    # scalp.umapwrap.graph_umap(dataset,grap,label = 'umap', n_components = dim)
    scalp.umapwrap.graph_pacmap(dataset,csr_matrix(grap).toarray(),label = 'umap', n_components = dim)
    return scalp.transform.stack(dataset)

for i in [1,2,3,4]:
    stack = Scalp(dataset[:i])
    sc.pl.umap(stack, color=['batch', 'label'])

In [ ]:
import ubergauss.tools as ut
from lmz import *

funcs = [scanorama, umaponly, bbknn, combat, Scalp] 

fuid = Range(funcs)
dataid = Range(datasets) 
tasks = [(f,d) for f in fuid for d in dataid]

def run(fd):
    f,d = fd
    fun = funcs[f]
    dat = datasets[d]
    stack = fun(dat,8)
    return stack
    
mydata = ut.xxmap(run, tasks) 

In [ ]:
from scalp.score import score_lin, score_lin_batch, score_scib_metrics
scoredics_lb = ut.xxmap(score_lin_batch, mydata)
scoredics_scib = ut.xxmap(score_scib_metrics, mydata)
scoredics_l = ut.xxmap(score_lin, mydata)

# score_lin_batch and score_lin -> pareto comparison

In [ ]:
import pandas as pd
import seaborn as sns
# this only applies if we dont use the scib score fucntion
funcs = 'scanorama, umaponly, bbknn, combat, Scalp'.split(', ')

results = [ {"method":funcs[f], 'score':s, 'dataset':d, 'target':'label' } for s,(f,d) in zip(scoredics_l,tasks)]
results += [ {"method":funcs[f], 'score':s, 'dataset':d, 'target':'batch'} for s,(f,d) in zip(scoredics_lb,tasks)]
df = pd.DataFrame(results)
sns.barplot(data=df, y = 'score', x = 'method', errorbar = 'sd', hue='target')
plt.show()
ours = df.pivot_table(index='method', columns='target', values='score')

In [ ]:
import lmz
# this is for SCIB scoring 


funcs = 'scanorama, umaponly, bbknn, combat, Scalp'.split(', ')
results = [ [{"method":funcs[f], 'score':ss, 'dataset':d, 'metric':scrmeth } 
             for (scrmeth,ss) in s.items() ]for s,(f,d) in zip(scoredicts,tasks)]
results = lmz.Flatten(results)

df = pd.DataFrame(results)
sns.barplot(data=df, y = 'score', x = 'method', errorbar = 'sd', hue = 'metric')
plt.legend(loc='right', bbox_to_anchor=(1.85, 0.5), ncol=1)
plt.show()

In [ ]:

def split_scib_scores(dicts):
    batchwords = 'PCR_batch ASW_label/batch graph_conn'.split()
    
    def split(d):
        b = np.mean([v for k,v in d.items() if k in batchwords ])
        a = np.mean([v for k,v in d.items() if not k in batchwords ])
        return a,b
        
    scores = lmz.Map(split, dicts) 
    return lmz.Transpose(scores)    
  

scr_l, scr_b = split_scib_scores(scoredicts)
results = [ {"method":funcs[f], 'score':s, 'dataset':d, 'target':'bioconservation_scib_avg' } for s,(f,d) in zip(scr_l,tasks)]
results += [ {"method":funcs[f], 'score':s, 'dataset':d, 'target':'batch_scib_avg'} for s,(f,d) in zip(scr_b,tasks)]
df = pd.DataFrame(results)
sns.barplot(data=df, y = 'score', x = 'method', errorbar = 'sd', hue='target')
plt.show()
theirs = df.pivot_table(index='method', columns='target', values='score')

In [ ]:
df

In [ ]:
from ubergauss.optimization import pareto_scores
pareto_scores(df)

In [ ]:
results

In [ ]:
np.corrcoef(ours.batch.values, theirs.batch_scib_avg.values)[0,1], np.corrcoef(ours.label.values, theirs.bioconservation_scib_avg.values)[0,1]